In [13]:
#모듈 임포트 및 데이터프레임 생성
import pandas as pd
import numpy as np
import datetime

df_mt = pd.read_excel('data/maintask.xlsx')
df_dt = pd.read_excel('data/date_std.xlsx')
df_mt.fillna('', inplace=True)

#date DB에서 merge 필요한 column만 추출
df_dt2 = df_dt[['DATE', 'WN_SERIAL NO', 'STD_CODE']]
#MMM-DD포멧으로 Column 생성하여 붙여넣기
df_dt2['unique_date'] = np.nan

#mmm-dd형식의 날짜 데이터 생성
i=0
while i < len(df_dt2):
    x = df_dt2.iloc[i,0].strftime('%b %d, %Y') #std 코드에 해당하는 날짜를 가져옴
    df_dt2.loc[i,['unique_date']] = x
    i+=1

#Main Task DB에 날짜 데이터 붙이기
df = pd.merge(df_mt, df_dt2, how='left', left_on='DUE DATE', right_on='DATE')


C:\Users\user\AppData\Local\Temp\ipykernel_20692\3725841922.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dt2['unique_date'] = np.nan


In [15]:
url_id = 'http://127.0.0.1:81/PJT_workstation/index.py?id='

In [28]:
def make_clickable(url, code):
    return f'<a href={url}>{code}</a>'
    #return f'{url}" rel="noopener noreferrer" target="_blank">{name}'

df['link'] = df.apply(lambda x: make_clickable(url_id+x['TASK_CODE'], x['TASK_CODE']), axis=1)


In [35]:
swn = input('출력할 주차를 선택하세요')

#주차에 해당하는 주단위 일정표 Frame 생성(empty)
df_thisweek = pd.DataFrame()

#wn 지정하고 날짜 리스트 생성
def get_cols(wn):
    cols = []
    wn = str(wn)
    for i in range(1,8):
        a = wn+'_'+str(i)
        x = df_dt.loc[df_dt['STD_CODE']==a,['DATE']]
        x2 = x.iloc[0,0].strftime('%b %d, %Y')
        cols.append(x2)
    return cols

#주차 선택하기
col_thisweek = get_cols(swn)

for day in col_thisweek:
    dfs = df.loc[df['unique_date']==day,['link']]  #해당 날짜에 맞는 업무코드를 가져오기 (DataFrame)
    dfs.columns = [day] #Column 명 날짜로 변경
    dfs.reset_index(drop=True, inplace=True) #기존 인덱스 제거하기
    df_thisweek = pd.concat([df_thisweek,dfs], axis=1) #주단위 일정표에 붙이기

#빈 인덱스 만큼 Nan으로 합치기
lst_num = len(df_thisweek) #현재 데이터 프레임 index 개수 가져오기
if lst_num < 6:
    while len(df_thisweek) <= 6:
        edf = pd.DataFrame(index=[len(df_thisweek)])
        df_thisweek = pd.concat([df_thisweek,edf], axis=0)

table = df_thisweek.to_html()
table = table.replace('&lt;','<')
table = table.replace('&gt;','>')
print(table)


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Nov 27, 2022</th>
      <th>Nov 28, 2022</th>
      <th>Nov 29, 2022</th>
      <th>Nov 30, 2022</th>
      <th>Dec 01, 2022</th>
      <th>Dec 02, 2022</th>
      <th>Dec 03, 2022</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NaN</td>
      <td><a href=http://127.0.0.1:81/PJT_workstation/index.py?id=GMDO-A-221109-03>GMDO-A-221109-03</a></td>
      <td>NaN</td>
      <td>NaN</td>
      <td><a href=http://127.0.0.1:81/PJT_workstation/index.py?id=GMDO-A-221125-01>GMDO-A-221125-01</a></td>
      <td><a href=http://127.0.0.1:81/PJT_workstation/index.py?id=GMDO-A-221123-02>GMDO-A-221123-02</a></td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>1</th>
      <td>NaN</td>
      <td><a href=http://127.0.0.1:81/PJT_workstation/index.py?id=GMDO-A-221111-01>GMDO-A-221111-01</a></td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
     

In [15]:
lst_num = len(df_thisweek) #현재 데이터 프레임 index 개수 가져오기
if lst_num < 6:
    edf = pd.DataFrame(index=[lst_num,6])
    df_thisweek = pd.concat([df_thisweek,edf], axis=0)
display(df_thisweek)

,Nov 27,Nov 28,Nov 29,Nov 30,Dec 01,Dec 02,Dec 03
0,NaN,GMDO-A-221109-03,GMDO-A-211109-01,GMDO-A-211018-01,GMDO-A-211126-04,GMDO-A-211116-05,GMDO-A-211126-05
1,NaN,GMDO-A-221111-01,GMDO-A-211117-01,GMDO-A-211112-02,GMDO-A-221125-01,GMDO-A-211124-01,GMDO-A-211201-05
2,NaN,GMDO-A-221122-02,GMDO-A-211122-03,GMDO-A-211126-03,NaN,GMDO-A-221123-02,NaN
3,NaN,GMDO-A-221124-01,GMDO-A-211126-01,NaN,NaN,NaN,NaN
4,NaN,NaN,GMDO-A-211126-02,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
from IPython.display import HTML
df = pd.DataFrame({
    'name':['Softhints', 'DataScientyst'],
    'url':['https://www.softhints.com', 'https://datascientyst.com']
})
df.style

,name,url
0,Softhints,https://www.softhints.com
1,DataScientyst,https://datascientyst.com


In [12]:
def make_clickable(url, name):
    return '<a href="{}">{}</a>'.format(url, name)
    #return f'{url}" rel="noopener noreferrer" target="_blank">{name}'

df['link'] = df.apply(lambda x: make_clickable(x['url'], x['name']), axis=1)
df.style

,name,url,link
0,Softhints,https://www.softhints.com,Softhints
1,DataScientyst,https://datascientyst.com,DataScientyst


In [32]:
#주차에 해당하는 주단위 일정표 Frame 생성(empty)
df_thisweek = pd.DataFrame()
print(df_thisweek)

Empty DataFrame
Columns: []
Index: []


In [4]:
#wn 지정하고 날짜 리스트 생성
def get_cols(wn):
    cols = []
    wn = str(wn)
    for i in range(1,8):
        a = wn+'_'+str(i)
        x = df_dt.loc[df_dt['STD_CODE']==a,['DATE']]
        x2 = x.iloc[0,0].strftime('%b %d')
        cols.append(x2)
    return cols

col_thisweek = get_cols(420)
print(col_thisweek)

['Nov 27', 'Nov 28', 'Nov 29', 'Nov 30', 'Dec 01', 'Dec 02', 'Dec 03']


In [24]:
a= df.loc[df['mmm-dd']==col_thisweek[1],['TASK_CODE']]
a.columns = [col_thisweek[1]]
a = a.reset_index(drop=True)
print(a)
print(col_thisweek)


             Nov 28
0  GMDO-A-221109-03
1  GMDO-A-221111-01
2  GMDO-A-221122-02
3  GMDO-A-221124-01
['Nov 27', 'Nov 28', 'Nov 29', 'Nov 30', 'Dec 01', 'Dec 02', 'Dec 03']


In [33]:
#날짜별로 리스트 가져와서 일정표 프레임에 붙이기.
df_thisweek.reset_index(drop=True, inplace=True)
for day in col_thisweek:
    dfs = df.loc[df['mmm-dd']==day,['TASK_CODE']]  #해당 날짜에 맞는 업무코드를 가져오기 (DataFrame)
    dfs.columns = [day] #Column 명 날짜로 변경
    dfs.reset_index(drop=True, inplace=True)
    df_thisweek = pd.concat([df_thisweek,dfs], axis=1) #주단위 일정표에 붙이기
print(df_thisweek)



  Nov 27            Nov 28            Nov 29            Nov 30  \
0    NaN  GMDO-A-221109-03  GMDO-A-211109-01  GMDO-A-211018-01   
1    NaN  GMDO-A-221111-01  GMDO-A-211117-01  GMDO-A-211112-02   
2    NaN  GMDO-A-221122-02  GMDO-A-211122-03  GMDO-A-211126-03   
3    NaN  GMDO-A-221124-01  GMDO-A-211126-01               NaN   
4    NaN               NaN  GMDO-A-211126-02               NaN   

             Dec 01            Dec 02            Dec 03  
0  GMDO-A-211126-04  GMDO-A-211116-05  GMDO-A-211126-05  
1  GMDO-A-221125-01  GMDO-A-211124-01  GMDO-A-211201-05  
2               NaN  GMDO-A-221123-02               NaN  
3               NaN               NaN               NaN  
4               NaN               NaN               NaN  


In [4]:
try: df
except NameError: df = None

In [8]:
df = None

In [10]:
import set_up

try: df
except NameError: df = None

if df is None:
    df = set_up.dbsetting()

print(df.head(5))

content-type: text/html; charset=euc-kr



c:\Bitnami\wampstack-8.1.11-0\apache2\htdocs\PJT_workstation\set_up.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dt2['unique_date'] = np.nan


   PROCESS         TASK_CODE                              TASK_NAME COVERAGE  \
0  DECLINE  BGTC-B-210222-01             바르게 살기 위원회 월간회의 개최_DECLINE   동 주민센터   
1       완료  BGTC-B-210222-02  직능단체 연락처 통합 정리 및 메시지 발송 폼 작성_COMPLETE   동 주민센터   
2       완료  CDTC-B-210222-02      민방위 편성 자원 전체 통합 관리 파일 작성_COMPLETE   동 주민센터   
3       완료  GTCH-S-210223-02    서울시장 보궐선거 선거인명부 확인 및 구청 방문_COMPLETE      서울시   
4       완료  GTCH-S-210223-03              주민센터 4층에서 공보물 작업_COMPLETE      서울시   

        PRIORITY   DUE DATE CTG_1  CTG_2  CTG_3 NAME   TEL             HP ETC  \
0       Not Busy 2021-02-23   은평구   주민센터    대조동  선호원  5184  010-3304-0269       
1       Not Busy 2021-02-23   은평구   주민센터    대조동  선호원  5184  010-3304-0269       
2       Not Busy 2021-02-26   은평구  행정안전국  자치안전과  선호원  5184  010-3304-0269       
3  Prior Concern 2021-03-20   은평구  구청 직할  의회사무국  선관위     -              -       
4  Prior Concern 2021-03-27   은평구  구청 직할  의회사무국  선관위     -              -       

  OFFICIAL_DOCS_NO         RECOR